In [2]:
import pandas as pd
import sys
sys.path.append("..")
from src.funcs import *
from src.query_funcs import *
from src.query_text import *
import dotenv
dotenv.load_dotenv()

True

In [ ]:
# Término de búsqueda
search = "Pasta recipe #cooking"
pasta_recipes = generate_results(search)

search = "#cooking"
general_recipes = generate_results(search)

search = "Chicken recipe #cooking"
chicken_recipes = generate_results(search)

search = "Vegan recipe #cooking"
vegan_recipes = generate_results(search)

search = "Chinese recipe #cooking"
chinese_recipes = generate_results(search)

100%|██████████| 71/71 [01:26<00:00,  1.22s/it]


In [ ]:
# pasta_recipes.to_csv("datos/pasta.csv")
# general_recipes.to_csv("datos/general.csv")
# chicken_recipes.to_csv("datos/chicken.csv")
# vegan_recipes.to_csv("datos/vegan.csv")
# chinese_recipes.to_csv("datos/chinese.csv")

In [14]:
pasta_recipes = pd.read_csv('datos/pasta.csv', index_col = 0)
general_recipes = pd.read_csv('datos/general.csv', index_col = 0)
chicken_recipes = pd.read_csv('datos/chicken.csv', index_col = 0)
vegan_recipes = pd.read_csv('datos/vegan.csv', index_col = 0)
chinese_recipes = pd.read_csv('datos/chinese.csv', index_col = 0)

In [15]:
pasta_titles = clean_texts(pasta_recipes['title'])
general_titles = clean_texts(general_recipes['title'])
chicken_titles = clean_texts(chicken_recipes['title'])
vegan_titles = clean_texts(vegan_recipes['title'])
chinese_titles = clean_texts(chinese_recipes['title'])

In [16]:
print("Pasta Titles:", pasta_titles[:2])
print("General Titles:", general_titles[:3])
print("Chicken Titles:", chicken_titles[:2])
print("Vegan Titles:", vegan_titles[:2])
print("Chinese Titles:", chinese_titles[:2])

Pasta Titles: ['parmesan cream pasta', 'how much pasta should you cook']
General Titles: ['dragon s breath with', 'the real spaghetti o s', 'how to cook with ice']
Chicken Titles: ['marry me chicken flavour fusion chicken', 'the best chicken']
Vegan Titles: ['annoying the vegans', 'high protein best vegan patties without excessive calories batch cook freeze and add to any meal']
Chinese Titles: ['they call this drug eggs in korea because these are so addictive day 10 of korean food', 'restaurant style chinese broccoli']


In [17]:
pasta_urls = obtener_links_paralelos(pasta_titles[:5])
general_urls = obtener_links_paralelos(general_titles[:5])
chicken_urls = obtener_links_paralelos(chicken_titles[:5])
vegan_urls = obtener_links_paralelos(vegan_titles[:5])
chinese_urls = obtener_links_paralelos(chinese_titles[:5])

cannot access local variable 'url' where it is not associated with a value
cannot access local variable 'url' where it is not associated with a value
cannot access local variable 'url' where it is not associated with a value
cannot access local variable 'url' where it is not associated with a value


In [20]:
general_urls

['https://tasty.co/recipe/dragon-mother',
 'https://www.allrecipes.com/recipe/11885/moms-best-spaghetti-sauce/',
 'https://www.allrecipes.com/recipe/8368641/deconstructed-big-mac/',
 'https://www.allrecipes.com/recipe/233928/how-to-make-vanilla-ice-cream/',
 'https://www.allrecipes.com/recipe/237678/kfc-style-spice-for-chicken/']

In [21]:
list_recipes_social = [
    pasta_recipes.iloc[:5],
    general_recipes.iloc[:5],
    chicken_recipes.iloc[:5],
    vegan_recipes.iloc[:5],
    chinese_recipes.iloc[:5]
    ]

In [24]:
url_lists = [
    (pasta_urls, 'pasta'),
    (general_urls, 'general'),
    (chicken_urls, 'chicken'),
    (vegan_urls, 'vegan'),
    (chinese_urls, 'chinese')
]

df_ingredients = pd.DataFrame()
i = 0
for url_list, recipe_type in url_lists:
    socials = list_recipes_social[i]
    j = 0
    for url in url_list:
        if not url:
            continue
        elif 'tasty.co' in url:
            df_it = tasty_ing(url)
        elif 'allrecipes.com' in url:
            df_it = allrecipes_ing(url)

        df_it['recipe_type'] = recipe_type
        df_it['link'] = url
        df_it['views'] = socials['views'].values[j]
        df_it['date'] = socials['date'].values[j]

        df_ingredients = pd.concat([df_ingredients, df_it], ignore_index=True)
        j += 1
    i += 1


## EDAMAM API

In [26]:
ingredients = edamam_query(df_ingredients[["amount", "unit", "ingredient"]])

In [28]:
df_final = pd.DataFrame()
for title in df_ingredients["title"].unique():
    recipe_df = df_ingredients[df_ingredients["title"] == title]
    ingredients = edamam_query(recipe_df[["amount", "unit", "ingredient"]])
    try:
        nutrients_df = get_nutrients(ingredients, float(recipe_df["servings"].unique()[0]))
    except Exception as e:
        print(ingredients)
        print(e)
    nutrients_df["title"] = title
    recipe_type = df_ingredients[df_ingredients["title"] == title]["recipe_type"].unique()[0]
    recipe_url = df_ingredients[df_ingredients["title"] == title]["link"].unique()[0]
    recipe_views = df_ingredients[df_ingredients["title"] == title]["views"].unique()[0]
    recipe_date = df_ingredients[df_ingredients["title"] == title]["date"].unique()[0]
    nutrients_df["recipe_type"] = recipe_type
    nutrients_df["recipe_url"] = recipe_url
    nutrients_df["views"] = recipe_views
    nutrients_df["date"] = recipe_date
    df_final = pd.concat([df_final, nutrients_df])

In [54]:
df_recipes = df_final.groupby('title').agg({
    'Calories (kcal)': 'sum',
    'Protein (g)': 'sum',
    'Fat (g)': 'sum',
    'Carbohydrates (g)': 'sum',
    'Sugar (g)': 'sum',
    'Fiber (g)': 'sum',
    'Serving weight (g)': 'first',  # Asumimos que el peso por porción es el mismo para toda la receta
    'recipe_type' : 'first',
    'recipe_url' : 'first',
    'views' : 'first',
    'date' : 'first'
}).reset_index()

In [55]:
df_recipes.head()

,title,Calories (kcal),Protein (g),Fat (g),Carbohydrates (g),Sugar (g),Fiber (g),Serving weight (g),recipe_type,recipe_url,views,date
0,Baked Pasta,571.580610,29.328934,27.558452,51.034106,6.279363,3.518768,281.729937,pasta,https://www.allrecipes.com/recipe/11906/baked-...,1879803,2024-09-12 11:14:32
1,"Chicken French (Rochester, NY Style)",598.143216,66.375730,28.917675,12.661439,4.411881,0.309421,359.184700,chicken,https://www.allrecipes.com/recipe/213487/chick...,1028296,2024-02-29 04:45:03
2,Classic Old Fashioned,160.013333,0.199000,0.034500,14.126533,13.489700,0.640000,322.550491,chinese,https://www.allrecipes.com/recipe/162397/class...,5194650,2024-08-14 04:00:14
3,Deconstructed Big Mac,44.200583,0.089818,3.724229,2.892519,2.302262,0.056698,13.927083,general,https://www.allrecipes.com/recipe/8368641/deco...,171646659,2024-04-01 08:25:24
4,Dragon Mother,825.692500,2.346025,2.356550,209.741375,201.468738,6.194500,476.595367,general,https://tasty.co/recipe/dragon-mother,208572055,2024-01-19 05:00:22


In [56]:
df_recipes['Health Score'] = df_recipes.apply(lambda row: calcular_puntuacion_salud_por_porcion(
    row['Protein (g)'], row['Carbohydrates (g)'], row['Fat (g)'], row['Fiber (g)'], row['Sugar (g)'], row['Calories (kcal)']
), axis=1)

In [57]:
df_recipes.head(2)

,title,Calories (kcal),Protein (g),Fat (g),Carbohydrates (g),Sugar (g),Fiber (g),Serving weight (g),recipe_type,recipe_url,views,date,Health Score
0,Baked Pasta,571.580610,29.328934,27.558452,51.034106,6.279363,3.518768,281.729937,pasta,https://www.allrecipes.com/recipe/11906/baked-...,1879803,2024-09-12 11:14:32,110.734584
1,"Chicken French (Rochester, NY Style)",598.143216,66.375730,28.917675,12.661439,4.411881,0.309421,359.184700,chicken,https://www.allrecipes.com/recipe/213487/chick...,1028296,2024-02-29 04:45:03,127.916372


In [36]:
unique_ings = df_final.Ingredient.unique()
# pd.DataFrame(unique_ings)
ings_100g = [f'100 g of {e}' for e in unique_ings]
df_concat = pd.DataFrame()
batch_size = 30
for i in range(0, len(ings_100g), batch_size):
    df_it = get_nutrients(ings_100g[i:i + batch_size], 1)
    df_concat = pd.concat([df_concat, df_it])
df_concat = df_concat.drop(columns=["Weight (g)", "Serving weight (g)"])

In [58]:
df_concat.head()

,Ingredient,Calories (kcal),Protein (g),Fat (g),Carbohydrates (g),Sugar (g),Fiber (g)
0,cooking spray,792.0,0.26,78.7,20.70,0.00,0.0
1,ground beef,254.0,17.20,20.0,0.00,0.00,0.0
2,tomato sauce,24.0,1.20,0.3,5.31,3.56,1.5
3,half-and-half,131.0,3.13,11.5,4.30,4.13,0.0
4,Parmesan cheese,392.0,35.80,25.0,3.22,0.11,0.0


In [40]:
crear_db('recetas')

Base de datos recetas creada con éxito


In [ ]:
conexion = establecer_conn("recetas", "admin", "postgres")
query_commit(conexion, create_tipos_table)

conexion = establecer_conn("recetas", "admin", "postgres")
query_commit(conexion, create_recetas_table)

conexion = establecer_conn("recetas", "admin", "postgres")
query_commit(conexion, create_ingredientes_table)

conexion = establecer_conn("recetas", "admin", "postgres")
query_commit(conexion, create_receta_ingredientes_table)

In [60]:
tipo_mapper = mapeo(df_recipes, "recipe_type")

In [61]:
lista_tuplas = [tuple([v,k]) for k,v in tipo_mapper.items()]
conexion = establecer_conn("recetas", "admin", "postgres")
query_commit_many(conexion, insert_tipos_query, lista_tuplas)

Done!


In [62]:
lista_tuplas = [tuple(e) for e in df_concat.values]
conexion = establecer_conn("recetas", "admin", "postgres")
query_commit_many(conexion, insert_ingrediente_query, lista_tuplas)

Done!


In [67]:
lista_tuplas = [tuple(e) for e in df_recipes.values]
conexion = establecer_conn("recetas", "admin", "postgres")
query_commit_many(conexion, insert_receta_query, lista_tuplas)

Done!


Hacemos una query para extraer los ids de cada uno de los ingredientes y recetas. Con esto podremos mapear los ingredientes de las recetas a unos ids específicos.

In [68]:
conexion = establecer_conn("recetas", "admin", "postgres")
ids_ingredientes = query_fetch(conexion, 'select ingredient_id, ingredient_name from Ingredientes')
ing_mapper = dict()
for ing in ids_ingredientes:
    ing_mapper[ing[1]] = ing[0]

In [69]:
df_final["Ingredient"] = df_final["Ingredient"].map(ing_mapper)

In [70]:
df_final["recipe_type"] = df_final["recipe_type"].map(tipo_mapper)

In [71]:
conexion = establecer_conn("recetas", "admin", "postgres")
ids_recipes = query_fetch(conexion, 'select recipe_id, title from Recetas')
recipe_mapper = dict()
for rec in ids_recipes:
    recipe_mapper[rec[1]] = rec[0]

In [72]:
df_final["title"] = df_final["title"].map(recipe_mapper)

In [73]:
df_receta_ingredientes = df_final.reindex(columns=["title", "Ingredient", "Weight (g)", "Calories (kcal)", "Protein (g)", "Fat (g)", "Carbohydrates (g)", "Sugar (g)", "Fiber (g)", "Serving weight (g)"])

In [74]:
df_receta_ingredientes.head(2)

,title,Ingredient,Weight (g),Calories (kcal),Protein (g),Fat (g),Carbohydrates (g),Sugar (g),Fiber (g),Serving weight (g)
0,1,1,3.780118,29.938531,0.009828,2.974953,0.782484,0.0,0.0,281.729937
1,1,2,56.699046,144.015577,9.752236,11.339809,0.000000,0.0,0.0,281.729937


In [75]:
lista_tuplas = [tuple(e) for e in df_receta_ingredientes.values]
conexion = establecer_conn("recetas", "admin", "postgres")
query_commit_many(conexion, insert_receta_ingredientes_query, lista_tuplas)

Done!
